In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [ ]:
train_dataset_path = "/content/dataset/chest_xray/chest_xray/train"

CONFIG = {
    "IM_SIZE":256,
    "BATCH_SIZE":32,
}

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dataset_path,
    image_size= (CONFIG["IM_SIZE"],CONFIG["IM_SIZE"]),
    batch_size=CONFIG["BATCH_SIZE"],
    seed = 123,
    shuffle= True,
    subset="training",
    validation_split = 0.2,
    label_mode = "binary",
)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dataset_path,
    image_size= (CONFIG["IM_SIZE"],CONFIG["IM_SIZE"]),
    batch_size=CONFIG["BATCH_SIZE"],
    seed = 123,
    shuffle= True,
    subset="validation",
    validation_split = 0.2,
    label_mode = "binary",
)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
plt.figure(figsize = (12,12))

for images,labels in train_ds.take(1):
  for i in range(16):
    plt.subplot(4,4,i+1)
    plt.title(class_names[int(labels[i].numpy()[0])])
    plt.imshow(images[i]/255)
    plt.axis("off")

In [ ]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal",input_shape=(CONFIG["IM_SIZE"],CONFIG["IM_SIZE"],3)),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
  ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
train_ds = train_ds.shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
print(f"Train Size: {len(train_ds)} Val Size: {len(val_ds)}")
print(train_ds)
print(val_ds)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape = (CONFIG["IM_SIZE"],CONFIG["IM_SIZE"],3)),
    data_augmentation,

    tf.keras.layers.Conv2D(filters = 32,kernel_size = 3,strides = 1,padding = "valid",activation = "relu"),
    tf.keras.layers.MaxPool2D(),

    tf.keras.layers.Conv2D(filters = 64,kernel_size = 3,strides = 1,padding = "valid",activation = "relu"),
    tf.keras.layers.MaxPool2D(),

    tf.keras.layers.Conv2D(filters = 128,kernel_size = 3,strides = 1,padding = "valid",activation = "relu"),
    tf.keras.layers.MaxPool2D(),

    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512,activation = "relu"),
    tf.keras.layers.Dense(100,activation = "relu"),

    tf.keras.layers.Dense(1,activation = "sigmoid")
])
model.summary()

In [ ]:
model.compile(optimizer='adam',loss = tf.keras.losses.binary_crossentropy,metrics = ['accuracy'])

In [ ]:
history = model.fit(train_ds,validation_data = val_ds,epochs = 10)

In [ ]:
model.evaluate(val_ds)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(["train_loss","val_loss"])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(["train_acc","val_acc"])
plt.show()

In [ ]:
plt.figure(figsize = (16,16))
text = ""
for images,labels in val_ds.take(1):
  for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(images[i]/255)
    true_label = class_names[int(labels[i].numpy()[0])]
    img_pred = tf.expand_dims(images[i],axis = 0)
    prediction = model.predict(img_pred)
    if prediction >= 0.5:
      text = "PNEUMONIA"
    else:
      text = "NORMAL"
    plt.title(f"True: {true_label} \n Prediction: {text}")
    plt.axis("off")